In [2]:
import re
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [27]:
service_words = ['void', 'int', 'float', 'char', 'return', 'goto', 'if',
                 'else', 'while', '{', '}', 'main', '=']
operations = ['<', '>', '<=', '>=', '==', '!=', '+', '-', '*', '/', '^', '||', '&&', '!']
separators = [' ', ',', ':', ';', '(', ')', '[', ']', '\'']

In [10]:
def filter_program(text):
    formatted_text = []
    lines = text.split('\n')
    for index, line in enumerate(lines):
        splitted_text = line.split('\'')
        even_flag = False
        for span in splitted_text:
            if not even_flag:
                formatted_span = re.sub(r'[\t\n]+', ' ', span)
                formatted_span = re.sub(r' +', ' ', formatted_span)
                formatted_span = re.sub(r'/\*.*\*/', '', formatted_span)
                formatted_text.append({
                    'text': formatted_span.lower(),
                    'type': 'code',
                    'line': index + 1
                    
                })
            else:
                formatted_text.append({
                    'text': span,
                    'type': 'string',
                    'line': index + 1
                })

            even_flag = not even_flag
        if not even_flag:
            print(Fore.RED + 'Unexpected end of line. There is unclosed apostrophe!' + Style.RESET_ALL)
            return None
    
#     formatted_text = formatted_text.replace('\\', '\\\\')

    return formatted_text
        

In [11]:
class Analyzer:
    state = 'S'
    string = ''
    collecting_string = ''
    
    def __init__(self, string):
        self.string = string
        
    def reset(self):
        self.collecting_string = ''
        self.state = 'S'
        
    def unexpected(self, symbol):
        self.state = 'error'
        print(Back.RED, Fore.WHITE,'ERRORE!', Style.RESET_ALL, 'Unexpected symbol \"' + symbol + '\"')
        return {
            'kind': 'error',
            'token': symbol,
            'residue': ''
        }
    
    def symbol_return(self, symbol):
        self.string = symbol + self.string
        if symbol != '':
            self.collecting_string = self.collecting_string[:-1]
    
    def collect_next(self):
#         print('String: "',  end='')
#         print(Fore.BLUE + self.string + Style.RESET_ALL, end='')
#         print('"')
        
        try:
            symbol = self.string[0]
            self.string = self.string[1:]
        except:
            symbol = ''
        self.collecting_string += symbol
        
        if self.state == 'S':
            if symbol.isalpha() or symbol == '_':
                self.state = 'letter_at_first'
            elif symbol == '<':
                self.state = '<_at_first'
            elif symbol == '>':
                self.state = '>_at_first'
            elif symbol in operations:
                return {
                    'kind': 'operation',
                    'token': symbol,
                    'residue': self.string
                }
            elif symbol.isdigit():
                self.state = 'digit_at_first'
            elif symbol == '.':
                self.state = '._at_first'
            return self.collect_next()
        
        if self.state in ['number -> .. -> number']:
            if symbol.isdigit():
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'integer_interval',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
        
        if self.state in ['number -> ..']:
            if symbol.isdigit():
                self.state = 'number -> .. -> number'
                return self.collect_next()
            else:
                return self.unexpected(symbol)
        
        if self.state == 'digit_at_first':
            if symbol.isdigit():
                return self.collect_next()
            elif symbol == '.':
                self.state = 'number -> .'
                return self.collect_next()
            elif symbol == 'e':
                self.state = 'number -> e'
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'integer',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
            
        if self.state == 'number -> .':
            if symbol == '.':
                self.symbol_return(symbol)
                self.symbol_return(symbol)
                return {
                    'kind': 'integer',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            
        if self.state == '._at_first':
            if symbol == '.':
                return {
                    'kind': 'separator',
                    'token': self.collecting_string,
                    'residue': self.string
                }
        
        if self.state in ['._at_first', 'number -> .']:
            if symbol.isdigit():
                return self.collect_next()
            elif symbol == 'e':
                self.state = 'number -> e'
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'real',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            elif symbol == '.':
                self.state = 'number -> ..'
                return self.collect_next()
            else:
                return self.unexpected(symbol)
                
        if self.state == 'number -> e':
            if symbol in ['+', '-']:
                self.state = 'number -> e -> +/-'
                return self.collect_next()
            elif symbol.isdigit():
                self.state = 'number -> e -> digit'
                return self.collect_next()
            else:
                return self.unexpected(symbol)
        
        if self.state in ['number -> e -> +/-', 'number -> e -> digit']:
            if symbol.isdigit():
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'real',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
                
        if self.state == '<_at_first':
            if symbol in ['>', '=']:
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                self.symbol_return(symbol)
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
        
        if self.state == '>_at_first':
            if symbol == '=':
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                self.symbol_return(symbol)
                return {
                    'kind': 'operation',
                    'token': self.collecting_string,
                    'residue': self.string
                }
        
        if self.state == 'letter_at_first':
            if symbol.isalpha() or symbol.isdigit() or symbol == '_':
                return self.collect_next()
            elif symbol in operations + ['']:
                self.symbol_return(symbol)
                return {
                    'kind': 'identifier',
                    'token': self.collecting_string,
                    'residue': self.string
                }
            else:
                return self.unexpected(symbol)
            
        return {
            'kind': 'exeption',
            'token': str(self.state),
            'residue': self.string
        }

In [12]:
def split_by_token(line):
    global separators
    
    splitter_index = len(line)
    for index, character in enumerate(line):
        for i in range(6, -1, -1):
            token = line[index:index+i]
            if token in separators + operations + service_words:
                splitter_index = index if index == 0 else index - 1
                return [line[:splitter_index+1], line[splitter_index+1:]]
    return [line[:splitter_index+1], line[splitter_index+1:]]

In [50]:
def get_next_token():
    if not segments:
        return False
    
    line = segments[0]['line']
    if segments[0]['type'] == 'code':
        [token, segments[0]['text']] = split_by_token(segments[0]['text'])
        if not segments[0]['text']:
            segments.pop(0)
            
        tables = [service_words, operations, separators, constants, identifiers]
        symbols = ['W', 'O', 'R', 'C', 'I']
        found = False

        for index, table in enumerate(tables):
            if token in table:
                return [symbols[index], table.index(token), line]
                found = True
                break

        if not found:
            automat = Analyzer(token)
            tail = token

            while tail:
                automat.reset()
                analyzed = automat.collect_next()
                kind = analyzed['kind']
                token_name = analyzed['token']
                tail = analyzed['residue']

                if kind == 'exeption':
                    return ERROR_xd
                else:
                    if kind == 'identifier':
                        identifiers.append({
                            'type': 'identifier',
                            'value': token
                        })
                        return ['I', len(identifiers) - 1, line]
                    if kind in ['string', 'integer', 'real']:
                        constants.append({
                            'type': kind,
                            'value': token
                        })
                        return ['C', len(constants) - 1, line]
    else:
        constants.append({
            'type': 'string',
            'value': segments[0]['text']
        })
        return ['C', len(constants) - 1, line]

In [65]:
segments = filter_program(open('./src/exp.cpp').read())
constants = []
identifiers = []
chain = []

token = 'nu sho, poihali'
while token:
    token = get_next_token()
    chain.append(token)
    print(token)
    
data = {
    'chain': chain[:-1],
    'tables': {
        'service_words': service_words,
        'operations': operations,
        'separators': separators,
        'constants': constants,
        'identifiers': identifiers
    }
}

with open('./res/lab1.json', 'w') as outfile:
    json.dump(data, outfile)

['I', 0, 1]
['R', 0, 1]
['W', 12, 1]
['R', 0, 1]
['I', 1, 1]
['R', 0, 1]
['O', 7, 1]
['R', 0, 1]
['I', 2, 1]
['R', 4, 1]
['I', 3, 1]
['R', 1, 1]
['R', 0, 1]
['I', 4, 1]
['R', 1, 1]
['R', 0, 1]
['I', 5, 1]
['O', 6, 1]
['C', 0, 1]
['R', 5, 1]
False
